Compléter le programme Python ci-dessous en répondant aux questions suivantes :
1. Ecrire une fonction qui retourne la position du pixel noir dans l’image.
2. Ajouter les instructions nécessaires afin de déplacer le pixel noir avec un pas (en nombre de
pixels) dans les quatre directions en tapant sur les touches 2, 4, 6 et 8. ('4' : Gauche, '6'
Droite, '8' : Haut, '2' : Bas) et '0' pour quitter le programme.
3. Afficher le pixel noir dans l’image avec un carré (5x5 pixels) sans l’utilisation des boucles
4. Sauvegarder l’image après le dernier déplacement avec un type de compression qui garde
l’image sans aucun changement.
5. Nous voulons afficher une forme (cercle, ellipse, croix ….) à la place du carré. Quel est le
filtre qui permet de réaliser cette opération et comment ? (donner des explications)

Explications des modifications :
1. Fonction find_black_pixel()
Utilise np.where() pour trouver les coordonnées du pixel noir

Retourne la position (y, x)

2. Déplacement avec les touches
Les touches '2', '4', '6', '8' déplacent le carré

'0' pour quitter

Pas de 5 pixels pour un déplacement visible

3. Carré 5x5 sans boucles
Utilise le slicing numpy : img[y_start:y_end, x_start:x_end] = 0

Gère les bords pour éviter les débordements

4. Sauvegarde sans perte
Format PNG avec cv2.imwrite()

Multiplie par 255 pour convertir [0,1] vers [0,255]

5. Réponse pour autres formes
Utiliser les fonctions de dessin d'OpenCV : cv2.circle(), cv2.ellipse(), cv2.line()

Plus efficace et plus flexible que les boucles manuelles

In [ ]:
import cv2
import numpy as np
from random import randrange

def createImgWithPointRand(h,w):
    img = np.ones((heightImg, widthImg), np.float32)
    # randrange(x) returns a random integer in the range [0, x)
    randPointY, randPointX = randrange(heightImg), randrange(widthImg)
    img[randPointY, randPointX] = 0
    return img

# 1. Fonction qui retourne la position du pixel noir
def find_black_pixel(img):
    positions = np.where(img == 0)
    if len(positions[0]) > 0:
        return positions[0][0], positions[1][0]  # y, x
    return None

# 3. Fonction pour afficher le pixel noir avec un carré 5x5 sans boucles
def draw_black_square(img, y, x, size=5):
    half = size // 2
    # Calcul des limites avec clipping pour éviter les débordements
    y_start = max(0, y - half)
    y_end = min(img.shape[0], y + half + 1)
    x_start = max(0, x - half)
    x_end = min(img.shape[1], x + half + 1)
    
    # Création du carré noir sans boucles
    img[y_start:y_end, x_start:x_end] = 0
    return img

# Fonction pour déplacer le pixel
def move_black_pixel(img, direction, step=1):
    pos = find_black_pixel(img)
    if pos is None:
        return img
    
    y, x = pos
    
    # Effacer l'ancienne position en remettant à 1 (blanc)
    img = np.ones((heightImg, widthImg), np.float32)
    
    # Calculer la nouvelle position
    if direction == '4':  # Gauche
        x = max(0, x - step)
    elif direction == '6':  # Droite
        x = min(widthImg - 1, x + step)
    elif direction == '8':  # Haut
        y = max(0, y - step)
    elif direction == '2':  # Bas
        y = min(heightImg - 1, y + step)
    
    # Dessiner le nouveau carré noir
    img = draw_black_square(img, y, x)
    return img, (y, x)

heightImg, widthImg = 200, 400
img = createImgWithPointRand(heightImg, widthImg)

# Trouver la position initiale et dessiner le carré
initial_pos = find_black_pixel(img)
if initial_pos:
    img = draw_black_square(img, initial_pos[0], initial_pos[1])

# Variables pour suivre la position
current_pos = initial_pos

while True:
    cv2.imshow('Image gray', img)
    
    # 2. Gestion des déplacements avec les touches
    q = cv2.waitKey(0) & 0xFF
    key = chr(q)
    
    if key == '0':  # Quitter
        break
    elif key in ['2', '4', '6', '8']:
        img, current_pos = move_black_pixel(img, key, step=5)
        print(f"Position actuelle: y={current_pos[0]}, x={current_pos[1]}")

cv2.destroyAllWindows()

# 4. Sauvegarder l'image avec compression sans perte
if current_pos:
    # PNG avec compression sans perte
    cv2.imwrite('images/image_finale.png', img * 255)  # Multiplier par 255 pour l'échelle 0-255
    print("Image sauvegardée avec compression PNG sans perte")

# 5. Réponse pour afficher d'autres formes
"""
Pour afficher une forme (cercle, ellipse, croix) à la place du carré, on peut utiliser:

FILTRE/MÉTHODE: Les fonctions de dessin d'OpenCV (cv2)

COMMENT:
1. Pour un cercle: cv2.circle()
   cv2.circle(img, (x, y), radius, color, thickness)

2. Pour une ellipse: cv2.ellipse()
   cv2.ellipse(img, (x, y), (axe_majeur, axe_mineur), angle, start_angle, end_angle, color, thickness)

3. Pour une croix: cv2.line() deux fois
   cv2.line(img, (x-5, y), (x+5, y), color, thickness)  # Ligne horizontale
   cv2.line(img, (x, y-5), (x, y+5), color, thickness)  # Ligne verticale

EXPLICATIONS:
- Ces fonctions permettent de dessiner des formes géométriques directement sur l'image
- Elles sont optimisées et plus efficaces que les boucles manuelles
- On contrôle la position, taille, couleur et épaisseur
- Pour notre cas, on remplacerait draw_black_square() par une de ces fonctions
- Exemple pour un cercle:
  cv2.circle(img, (x, y), 3, 0, -1)  # Cercle noir de rayon 3 pixels, rempli (-1)
"""